In [1]:
import pandas as pd
import numpy as np
from config import *
from feature_extract import *
from datetime import date

def extract_other_feature(dataset_file_path,num):
	print '第{0}个样本提取'.format(num)
	other_feature = pd.read_csv(dataset_file_path)
	# other_feature.drop_duplicates(inplace=True)
	print other_feature.shape
	# 用户该月/该天收到的优惠券数
	t1 = user_this_month_all_coupon_count(other_feature)
	t2 = user_this_day_all_coupon_count(other_feature)
	t3 = user_this_month_same_coupon_count(other_feature)
	t4 = user_this_day_same_coupon_count(other_feature)
	t5 = user_this_month_same_coupon_ago_late_date(other_feature)

	other_feature = pd.merge(other_feature,t1,on='User_id',how='left')
	other_feature = pd.merge(other_feature,t2,on=['User_id','Date_received'],how='left')
	other_feature = pd.merge(other_feature,t3,on=['User_id','Coupon_id'],how='left')

	other_feature = pd.merge(other_feature,t4,on=['User_id','Coupon_id','Date_received'],how='left')

	other_feature = pd.merge(other_feature,t5,on=['User_id','Coupon_id'],how='left')

	# 用户收到的优惠券是哪一天/周末吗
	other_feature['day_of_week'] = other_feature.Date_received.astype('str').apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1 if x!='null' else 'null')
	other_feature['is_weekend'] = other_feature.day_of_week.apply(lambda x:1 if x in (6,7) else 0 if x!='null' else 'null')
	other_feature['day_of_month'] = other_feature.Date_received.astype('str').apply(lambda x:int(x[6:8]) if x!='null' else 'null')

    
	other_feature.loc[:,'user_receive_coupon_first'] = other_feature.apply(lambda x:1 if (x['user_this_month_same_coupon_count']!=1)&(x['Date_received']==x['Date_received_min']) else 0,axis=1)
	other_feature.loc[:,'user_receive_coupon_last'] = other_feature.apply(lambda x:1 if (x['user_this_month_same_coupon_count']!=1)&(x['Date_received']==x['Date_received_max']) else 0,axis=1)
	other_feature['user_receive_coupon_first'][other_feature['user_this_month_same_coupon_count']==1] = -1
	other_feature['user_receive_coupon_last'][other_feature['user_this_month_same_coupon_count']==1] = -1

# 	other_feature = other_feature.drop(['Discount_rate'],axis=1)
	other_feature = other_feature.drop(['Date_received_min'],axis=1)
	other_feature = other_feature.drop(['Date_received_max'],axis=1)

	other_feature['discount_man'] = other_feature.Discount_rate.apply(get_discount_man)
	other_feature['discount_jian'] = other_feature.Discount_rate.apply(get_discount_jian)
	other_feature['is_man_jian'] = other_feature.Discount_rate.apply(is_man_jian)
	other_feature['discount_per'] = other_feature.Discount_rate.apply(calc_discount_rate)    
    
	print other_feature.shape

	other_feature.to_csv('data/other_feature{0}.csv'.format(num),index=None)

for k,path in enumerate(processed_dataset_path):
    extract_other_feature(path,k+1)

第1个样本提取
(137167, 7)


feature_extract.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  t1['user_this_month_all_coupon_count'] = 1
feature_extract.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  t1['user_this_day_all_coupon_count'] = 1
/usr/local/Cellar/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

用户该月收到coupon数提取
用户该天收到coupon数提取
用户该月收到相同coupon数提取
用户该天收到相同coupon数提取


feature_extract.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  t1['user_this_day_same_coupon_count'] = 1
/usr/local/Cellar/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


用户该月收到相同coupon日期提取


/usr/local/Cellar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/Cellar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(137167, 20)
第2个样本提取
(258446, 7)
用户该月收到coupon数提取
用户该天收到coupon数提取
用户该月收到相同coupon数提取
用户该天收到相同coupon数提取
用户该月收到相同coupon日期提取
(258446, 20)
第3个样本提取
(113640, 6)
用户该月收到coupon数提取
用户该天收到coupon数提取
用户该月收到相同coupon数提取
用户该天收到相同coupon数提取
用户该月收到相同coupon日期提取
(113640, 19)


In [2]:
temp=pd.read_csv('data/other_feature1.csv')

In [3]:
temp.shape

(137167, 20)

In [4]:
temp

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,user_this_month_all_coupon_count,user_this_day_all_coupon_count,user_this_month_same_coupon_count,user_this_day_same_coupon_count,day_of_week,is_weekend,day_of_month,user_receive_coupon_first,user_receive_coupon_last,discount_man,discount_jian,is_man_jian,discount_per
0,1832624,3381,7610,200:20,0,20160429,null,1,1,1,1,5,0,29,-1,-1,200,20,1,0.90
1,163606,1569,5054,200:30,10,20160421,null,1,1,1,1,4,0,21,-1,-1,200,30,1,0.85
2,4061024,3381,7610,200:20,10,20160426,null,1,1,1,1,2,0,26,-1,-1,200,20,1,0.90
3,106443,450,3732,30:5,null,20160429,null,1,1,1,1,5,0,29,-1,-1,30,5,1,0.83
4,114747,1569,5054,200:30,9,20160426,null,1,1,1,1,2,0,26,-1,-1,200,30,1,0.85
5,6038208,1569,5054,200:30,0,20160417,null,1,1,1,1,7,1,17,-1,-1,200,30,1,0.85
6,623832,4442,7643,100:50,null,20160429,null,1,1,1,1,5,0,29,-1,-1,100,50,1,0.50
7,918744,7555,9871,30:5,null,20160415,null,1,1,1,1,5,0,15,-1,-1,30,5,1,0.83
8,127227,3621,2705,20:5,0,20160510,null,1,1,1,1,2,0,10,-1,-1,20,5,1,0.75
9,3548000,3381,7610,200:20,10,20160429,null,1,1,1,1,5,0,29,-1,-1,200,20,1,0.90
